# zfit binned

There are two main ways of looking at "binned fits"
- Either an analytic shape that could be fit unbinned but is fit to binned data *because of the datasize* (typical LHCb, Belle II,...)
- stacking template histograms from simulation to provide the shape and fit to binned data (typical CMS, ATLAS, some LHCb,...)

Some templated fits with uniform binning, no analytic components and specific morphing and constraints fit into the HistFactory model, implemented in [pyhf](https://github.com/scikit-hep/pyhf).
These fits make a large portion of CMS and ATLAS analyses.

zfit can, in principle, reproduce them too. However, it's comparably inefficient, a lot of code and slow. The main purpose is to support *anything that is beyond HistFactory*.

In [ ]:
import hist
import matplotlib.pyplot as plt
import numpy as np
import zfit
import zfit.z.numpy as znp  # numpy-like backend interface
from zfit import z  # backend, special functions

## Binned parts

zfit introduces binned equivalents to unbinned components and transformations from one to the other.
For example:
- `SumPDF` -> `BinnedSumPDF`
- `Data` -> `BinnedData`
- `UnbinnedNLL` -> `BinnedNLL`

There are converters and new, histogram specific PDFs and methods.

## From unbinned to binned

Let's start with an example, namely a simple, unbinned fit that we want to perform binned.

In [ ]:
normal_np = np.random.normal(loc=2., scale=3., size=10000)

obs = zfit.Space("x", limits=(-10, 10))

mu = zfit.Parameter("mu", 1., -4, 6)
sigma = zfit.Parameter("sigma", 1., 0.1, 10)
model_nobin = zfit.pdf.Gauss(mu, sigma, obs)

data_nobin = zfit.Data.from_numpy(obs, normal_np)

loss_nobin = zfit.loss.UnbinnedNLL(model_nobin, data_nobin)

In [ ]:
minimizer = zfit.minimize.Minuit()

In [ ]:
# make binned
nbins = 50
data = data_nobin.to_binned(nbins)
model = model_nobin.to_binned(data.space)
loss = zfit.loss.BinnedNLL(model, data)

In [ ]:
result = minimizer.minimize(loss)
print(result)

In [ ]:
result.hesse(name="hesse")

In [ ]:
result.errors(name="errors")

In [ ]:
print(result)

## Binned parts in detail

`to_binned` creates a binned (and `to_unbinned` an unbinned) version of objects. It takes a binned Space, a binning or (as above), an integer (in which case a uniform binning is created).

This creates implicitly a new, binned space.

In [ ]:
obs_binned_auto = data.space
print(obs_binned_auto)

In [ ]:
print(f"Binned obs binning: {obs_binned_auto.binning}, is_binned: {obs_binned_auto.is_binned}")
print(f"Unbinned obs binning:{obs.binning}, is_binned: {obs.is_binned}")

In [ ]:
bkg = np.random.exponential(scale=1000, size=100_000)
bkg
